In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
num_classes = 5
#train_samples = 3000 #sumoftrain
#test_samples = 670  #sumofvalidation


In [3]:
from keras.applications import inception_v3 as inc_net
base_model = inc_net.InceptionV3(weights='imagenet', include_top=False)
print("InceptionV3 loaded")

Using TensorFlow backend.


InceptionV3 loaded


In [4]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
# idk needs a loot of improvement, but just for testing


x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

#freezing all layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range = 0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set= train_datagen.flow_from_directory('./data/flower_photos/train/', target_size = (299,299), batch_size=16, class_mode='categorical')
val_set = test_datagen.flow_from_directory('./data/flower_photos/test/', target_size= (299,299), batch_size=16, class_mode='categorical')

Found 3000 images belonging to 5 classes.
Found 670 images belonging to 5 classes.


In [6]:
#for i, layer in enumerate(base_model.layers):
#   print(i, layer.name)

In [7]:
model.fit_generator(training_set, steps_per_epoch=100, epochs=3, validation_steps=25)

Epoch 1/3
 47/100 [=============>................] - ETA: 3:13 - loss: 11.3326 - acc: 0.2726

KeyboardInterrupt: 

In [ ]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(training_set, steps_per_epoch=300, epochs=5, validation_steps=50)

In [ ]:
# SAVE THE MODEL
# serialize model to JSON
from keras.models import model_from_json

#model_json = model.to_json()
#with open("model.json", "w") as json_file:
#    json_file.write(model_json)
# serialize weights to HDF5
#model.save_weights("model.h5")
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
print("Saved model to disk")